In [1]:
import pandas as pd  # Pandas proporciona estructuras de datos y herramientas para el análisis de datos.
import seaborn as sns  # Seaborn es una biblioteca de visualización de datos basada en matplotlib.
import matplotlib.pyplot as plt  # Matplotlib es una biblioteca de gráficos 2D que produce figuras de calidad de publicación.
from pathlib import Path  # Pathlib ofrece clases para manejar rutas de archivos de manera orientada a objetos.
import os  # La biblioteca os permite interactuar con el sistema operativo.

In [2]:
s_path = Path.cwd()  # Obtiene el directorio de trabajo actual como un objeto Path.
s_prt_path = s_path.parent/'..'/'Datasets' # Accede al directorio padre del directorio actual. Utilizamos '..' para subir un nivel en la jerarquía de directorios.
ruta_carpeta_atras = s_prt_path.resolve() # Normalizamos la ruta para resolver cualquier ruta relativa
ruta_csv = os.path.join(s_prt_path,'DataPrepandemia_XM.csv')  # Crea una ruta al archivo CSV en el directorio padre.
df_DataModel= pd.read_csv(ruta_csv)  # Lee el archivo CSV en un DataFrame de pandas.

In [3]:
df_DataModel['Date'] = pd.to_datetime(df_DataModel['Date'])  # Convierte la columna 'Date' a tipo datetime.
df_DataModel.drop('Unnamed: 0', axis=1, inplace=True)  # Elimina la columna 'Unnamed: 0'.

In [4]:
df_DataModel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1461 non-null   datetime64[ns]
 1   Demanda    1461 non-null   float64       
 2   VolUtil    1461 non-null   float64       
 3   Aportes    1461 non-null   float64       
 4   PrecioB    1461 non-null   float64       
 5   PrecioO    1461 non-null   float64       
 6   DispTer    1461 non-null   float64       
 7   DispNoTer  1461 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 91.4 KB


In [5]:
from sklearn.ensemble import RandomForestRegressor

In [6]:
# Definir la ventana de tiempo (por ejemplo, 3 días)
ventana_tiempo = 2

In [7]:
# Crear características basadas en la ventana de tiempo para 'PrecioB'
for i in range(ventana_tiempo):
    df_DataModel[f'PrecioB_lag_{i+1}'] = df_DataModel['PrecioB'].shift(i+1)

In [8]:
# Eliminar las filas con valores NaN creados por el shift
df_DataModel = df_DataModel.dropna()

In [9]:
# Definir las variables independientes (X) y la variable dependiente (y)
X = df_DataModel.drop(['PrecioB', 'Date'], axis=1)
y = df_DataModel['PrecioB']

In [10]:
X.head(10)

,Demanda,VolUtil,Aportes,PrecioO,DispTer,DispNoTer,PrecioB_lag_1,PrecioB_lag_2,PrecioB_lag_3,PrecioB_lag_4,PrecioB_lag_5,PrecioB_lag_6,PrecioB_lag_7
7,7.665034e+06,1.035877e+10,56916800.0,430.419115,3.811161e+06,9.135660e+06,629.537537,633.454963,573.129733,535.140967,499.329203,407.627913,407.565972
8,7.319285e+06,1.031524e+10,58531700.0,453.829644,3.595526e+06,9.121233e+06,587.248523,629.537537,633.454963,573.129733,535.140967,499.329203,407.627913
9,6.542415e+06,1.028174e+10,55702100.0,449.891662,3.671894e+06,9.247313e+06,496.745422,587.248523,629.537537,633.454963,573.129733,535.140967,499.329203
10,6.639139e+06,1.024622e+10,55021300.0,464.874644,3.683867e+06,9.167086e+06,533.020900,496.745422,587.248523,629.537537,633.454963,573.129733,535.140967
11,7.592810e+06,1.018983e+10,51698900.0,448.358021,3.690565e+06,8.985126e+06,540.924392,533.020900,496.745422,587.248523,629.537537,633.454963,573.129733
12,7.757863e+06,1.013111e+10,57472800.0,461.912549,3.560153e+06,8.934978e+06,659.949268,540.924392,533.020900,496.745422,587.248523,629.537537,633.454963
13,7.834628e+06,1.007228e+10,47998200.0,458.685795,3.791296e+06,8.685103e+06,685.154920,659.949268,540.924392,533.020900,496.745422,587.248523,629.537537
14,7.903700e+06,1.001185e+10,49197100.0,441.016662,3.797674e+06,8.717157e+06,770.768537,685.154920,659.949268,540.924392,533.020900,496.745422,587.248523
15,7.438146e+06,9.961400e+09,48121500.0,439.755908,3.816076e+06,9.146723e+06,693.806823,770.768537,685.154920,659.949268,540.924392,533.020900,496.745422
16,6.739229e+06,9.928000e+09,45333900.0,443.534210,4.095417e+06,9.047262e+06,621.637368,693.806823,770.768537,685.154920,659.949268,540.924392,533.020900


In [11]:
# Dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Inicializar el modelo de Random Forest
modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [13]:
# Entrenar el modelo
modelo_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
# Realizar predicciones
y_predict = modelo_rf.predict(X_test)

In [15]:
# Evaluar el modelo
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_predict)
print(f'Error cuadrático medio: {mse}')

Error cuadrático medio: 427.7225194024815
